In [1]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
說明：
    SVD對照模型，直接連資料庫的，不需要事先準備資料

注意事項：
    資料庫欄位名稱是否一致
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

# 先import會用到的庫
from pymongo import MongoClient
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import os
import os.path
import math
from surprise import (Reader, Dataset, NormalPredictor, BaselineOnly,
                      accuracy, SVD, SVDpp, NMF, KNNBasic, KNNWithMeans,
                      KNNWithZScore, KNNBaseline)
from surprise.model_selection import (cross_validate, KFold,
                                      train_test_split, GridSearchCV,
                                      RandomizedSearchCV)

client = MongoClient('localhost', 27017)
db_final = client.Yelp_Final
review_final = db_final.review

In [2]:
# 讀取資料
checkins = dict()
bar = tqdm(total=review_final.count_documents({}), desc='Get Checkins')
tempIds = review_final.find({}, no_cursor_timeout=True, batch_size=10)
for item in tempIds:
    if item['newUserId'] not in checkins:
        checkins[item['newUserId']] = list()
        checkins[item['newUserId']].append([item['newBusinessId'], item['stars']])
    else:
        checkins[item['newUserId']].append([item['newBusinessId'], item['stars']])
    bar.update(1)
tempIds.close()
bar.close()

Get Checkins:   0%|          | 0/61189 [00:00<?, ?it/s]c:\Users\Yaote\anaconda3\envs\RoBERTa_pt\Lib\site-packages\pymongo\collection.py:1685: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)
Get Checkins: 100%|██████████| 61189/61189 [00:05<00:00, 11917.51it/s]


In [3]:
# 轉換成DataFrame並拆分成trainset和testset
newData = list()
for key in checkins:
    for poi, stars in checkins[key]:
        newData.append([key, poi, stars])

columns = ['userid', 'businessid', 'stars']
df = pd.DataFrame(newData, columns=columns)

reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df, reader)
trainset, testset = train_test_split(data, test_size=0.3)

In [4]:
# SVD++ (defaults)
svdpp = SVDpp()
svdpp.fit(trainset)
predictions = svdpp.test(testset)
print(accuracy.mae(predictions))
print(accuracy.rmse(predictions))

MAE:  0.8318
0.8318393697477332
RMSE: 1.0740
1.0739678756550952
